In [1]:
!kaggle datasets download -d olistbr/brazilian-ecommerce


Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0




  0%|          | 0.00/42.6M [00:00<?, ?B/s]
  2%|2         | 1.00M/42.6M [00:00<00:31, 1.39MB/s]
  5%|4         | 2.00M/42.6M [00:00<00:15, 2.80MB/s]
  9%|9         | 4.00M/42.6M [00:00<00:06, 6.07MB/s]
 19%|#8        | 8.00M/42.6M [00:01<00:02, 13.0MB/s]
 28%|##8       | 12.0M/42.6M [00:01<00:01, 18.6MB/s]
 38%|###7      | 16.0M/42.6M [00:01<00:01, 23.1MB/s]
 47%|####6     | 20.0M/42.6M [00:01<00:00, 26.8MB/s]
 56%|#####6    | 24.0M/42.6M [00:01<00:00, 28.4MB/s]
 66%|######5   | 28.0M/42.6M [00:01<00:00, 30.5MB/s]
 75%|#######5  | 32.0M/42.6M [00:01<00:00, 32.3MB/s]
 84%|########4 | 36.0M/42.6M [00:01<00:00, 33.6MB/s]
 94%|#########3| 40.0M/42.6M [00:02<00:00, 34.5MB/s]
100%|##########| 42.6M/42.6M [00:02<00:00, 21.2MB/s]
